In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' # Gets rid of SettingWithCopy warnings

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact

## Load Forecast Data

In [2]:
people = pd.read_csv('../data/forecast/people.csv', 
                          index_col='id', 
                          parse_dates=['updated_at'],
                          infer_datetime_format=True)

people.shape

(26, 22)

In [3]:
projects = pd.read_csv('../data/forecast/projects.csv', 
                          index_col='id', 
                          parse_dates=['updated_at','start_date','end_date'],
                          infer_datetime_format=True)

projects.shape

(90, 12)

In [4]:
placeholders = pd.read_csv('../data/forecast/placeholders.csv', 
                          index_col='id',
                          parse_dates=['updated_at'],
                          infer_datetime_format=True)

placeholders.shape

(8, 5)

In [5]:
clients = pd.read_csv('../data/forecast/clients.csv', 
                          index_col='id', 
                          parse_dates=['updated_at'],
                          infer_datetime_format=True)

clients.shape

(29, 5)

In [6]:
assignments = pd.read_csv('../data/forecast/assignments.csv', 
                          index_col='id', 
                          parse_dates=['start_date','end_date','updated_at'],
                          infer_datetime_format=True)

assignments['allocation'] = assignments['allocation']/(8*60*60)
assignments.shape

(177, 11)

## Functions to merge and display data on people's allocations

In [7]:
def get_person_id(first_name, last_name=None):
    if last_name is None:
        person_id = people.loc[(people['first_name']==first_name)]
    
        if len(person_id) != 1:
            raise ValueError('Could not unique person with name '+first_name)

    else:
        person_id = people.loc[(people['first_name']==first_name) & (people['last_name']==last_name)]
    
        if len(person_id) != 1:
            raise ValueError('Could not unique person with name '+first_name+' '+last_name)

    return person_id.index[0]


def get_person_allocations(person_id):    
    person_assignments = assignments[assignments['person_id']==person_id]
    
    start_date = projects['start_date'].min()
    end_date = projects['end_date'].max()

    weeks = pd.date_range(start=start_date, end=end_date, freq='MS')
    df = pd.DataFrame(index=weeks, columns=projects.index)
    df.fillna(0,inplace=True)
    
    for idx,row in person_assignments.iterrows():
        dates = pd.date_range(start=row['start_date'], end=row['end_date'], freq='MS')
        df.loc[dates, row['project_id']] = row['allocation']
             
    df.columns.name = people.loc[person_id,'first_name'] + ' ' + people.loc[person_id,'last_name']
    
    return df


def filter_person_allocations(allocations, start_date, end_date):    
    mask = (allocations.index >= start_date) & (allocations.index <= end_date)
    filtered_alloc = allocations.loc[mask]
    
    active_projects = filtered_alloc.columns[filtered_alloc.sum()!=0]
    filtered_alloc = filtered_alloc[active_projects]
    
    # replace project ids with project names
    name = filtered_alloc.columns.name
    filtered_alloc.columns = projects.loc[filtered_alloc.columns, 'name']
    filtered_alloc.columns.name = name
    
    return filtered_alloc


def plot_person_allocations(allocations, title=None):
    ax = plt.figure(figsize=(15,5)).gca()
    
    allocations.plot.area(ax=ax, linewidth=0)

    xlim = ax.get_xlim()
    plt.plot(xlim,[0.8, 0.8], 'k--', linewidth=3)
    plt.plot(xlim,[1, 1], 'k', linewidth=3)
    plt.xlim(xlim)
    plt.ylim([0,max([1, 1.1*allocations.sum(axis=1).max()])])

    plt.title(title)
    plt.ylabel('Proportion 8hr days')
    plt.xticks()
    plt.legend(title='')
    plt.show()


def highlight_allocs(s):
    is_over = s>1
    is_marginal = (s>0.8) & (is_over==False)
    is_under = s<0.8
    
    style = []
    for i in range(len(s)):
        if is_over[i]:
            style.append('background-color: red')
        elif is_marginal[i]:
            style.append('background-color: orange')
        elif is_under[i]:
            style.append('background-color: green')
        else:
            style.append('')
        
    return style

def person_allocations(first_name, last_name=None, 
                       start_date=pd.datetime(2018,1,1), end_date=pd.datetime(2020,1,1)):
    
    person_id = get_person_id(first_name, last_name)
    alloc = get_person_allocations(person_id)
    alloc_filt = filter_person_allocations(alloc, start_date, end_date)
    
    rows,cols = alloc_filt.shape
    if rows>0 and cols>0:
        plot_person_allocations(alloc_filt, 
                                title=people.loc[person_id,'first_name']+' '+people.loc[person_id,'last_name'])

        
        alloc_filt['TOTAL'] = alloc_filt.sum(axis=1)
        display(pd.DataFrame(alloc_filt, 
                             index=alloc_filt.index.strftime("%Y-%m-%d")).style.apply(highlight_allocs, 
                                                                                      subset=['TOTAL']))
        
    else:
        print('No data to plot.')
        
    return alloc_filt

def widget_person_alloc(Person):
    names = Person.split()
    person_allocations(names[0], ' '.join(names[1:]))

## Show Someone's Project Assignment

In [8]:
interact(widget_person_alloc, Person=sorted(people['first_name']+' '+people['last_name']));

interactive(children=(Dropdown(description='Person', options=('Amaani Hoddoon', 'Amber Raza', 'Angus Williams'…

To do:

Customisable date range

Customisable date frequency

Table summarising people under/over allocated

Project views